In [1]:
"""
Data Broker Clearinghouse Analysis
==================================
Exploratory analysis of data broker collection practices and transparency.
"""

import sys
import os
from pathlib import Path

import altair as alt
import pandas as pd
import numpy as np

# Add project root to path for imports
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Import custom functions
from data_utils.data_cleaner import clean_data

# Configuration
DATA_PATH = '../data/raw_data/Data_Broker_Full_Registry_2025.xlsx'
ALT_THEME = 'quartz'  # Set consistent theme for visualizations

# Set Altair theme for consistent styling
alt.data_transformers.enable('json')
alt.themes.enable(ALT_THEME)

from data_utils.privacy_policy_analyzer import (
    create_policy_analysis_chart
)

print("Loading and cleaning data...")
try:
    data_brokers = clean_data(DATA_PATH)
    print(f"Data loaded successfully! Shape: {data_brokers.shape}")
    print(f"Total brokers: {len(data_brokers)}")
except FileNotFoundError:
    print(f"Error: Could not find data file at {DATA_PATH}")
    print("Please check the file path and try again.")
except Exception as e:
    print(f"Error loading data: {e}")

# Display basic info
data_brokers.head()

/var/folders/dc/pxjd3wtj4_v88slzx9vq15740000gn/T/ipykernel_26917/1012559608.py:28: AltairDeprecationWarning: 
Deprecated since `altair=5.5.0`. Use altair.theme instead.
Most cases require only the following change:

    # Deprecated
    alt.themes.enable('quartz')

    # Updated
    alt.theme.enable('quartz')

If your code registers a theme, make the following change:

    # Deprecated
    def custom_theme():
        return {'height': 400, 'width': 700}
    alt.themes.register('theme_name', custom_theme)
    alt.themes.enable('theme_name')

    # Updated
    @alt.theme.register('theme_name', enable=True)
    def custom_theme():
        return alt.theme.ThemeConfig(
            {'height': 400, 'width': 700}
        )

See the updated User Guide for further details:
    https://altair-viz.github.io/user_guide/api.html#theme
    https://altair-viz.github.io/user_guide/customization.html#chart-themes
  alt.themes.enable(ALT_THEME)


Loading and cleaning data...
File saved to ../data/cleaned_data/uq-data-brokers.csv
Data loaded successfully! Shape: (941, 38)
Total brokers: 941


,Name,RegistrySource_California (Attorney General),RegistrySource_California (California Privacy Protection Agency),RegistrySource_Oregon,RegistrySource_Texas,RegistrySource_Vermont,CollectsNames,CollectsAddresses,CollectsDOB,CollectsPOB,...,County,DataCategories,Email,GroupUUID_Combined,OptOutURL,Phone,PrivacyPolicyURL,State,WebsiteURL,ZipCode
0,01ADVERTISING,False,True,False,True,True,2.0,2.0,2.0,2.0,...,None,"[""Not Minors Data""]",privacy@01advertising.com,01advertisingin_0_b87ca1b8,https://app.termly.io/notify/d1321341-0e82-4d8...,1.949610e+10,https://www.01advertising.com/legal/privacy/,CA,https://www.01advertising.com/,92660
1,180 BY TWO,False,False,False,False,True,2.0,2.0,2.0,2.0,...,None,"[""Not Minors Data""]",karl@180bytwo.com,180bytwollc_91cdd232,https://privacyportal.onetrust.com/webform/e32...,NaN,https://anteriad.com/privacy-policy,FL,https://www.180bytwo.com/,33755
2,33 MILE RADIUS,True,True,True,False,True,2.0,2.0,2.0,2.0,...,None,"[""Names"", ""Addresses"", ""Not DOB"", ""Not POB"", ""...",privacy@33mileradius.com,azarllc_1326078d,None,NaN,https://www.33mileradius.com/privacy-policy/,CO,https://www.33mileradius.com/,80205
3,33ACROSS,True,True,True,True,True,2.0,2.0,2.0,2.0,...,None,"[""Not Names"", ""Not Addresses"", ""Not DOB"", ""Not...",privacy@33across.com,33acrossinc_3_2a9457a6,https://udp.33across.com/udp_opt_out/cookie_op...,1.888297e+10,https://www.33across.com/privacy-policy/,NY,https://www.33across.com/,10003
4,360 MEDIA DIRECT,True,False,False,False,False,2.0,2.0,2.0,2.0,...,None,[],privacy@360mediadirect.com,subdirectllc_ce0c99b6,https://360-media-direct.privacy.saymine.io/36...,NaN,https://www.360mediadirect.com/privacy-policy,CA,https://www.360mediadirect.com/,93711


In [2]:
# Data Collection Analysis Functions
# ===================================

def analyze_collection_patterns(df, data_categories):
    """
    Analyze data collection patterns across multiple categories.
    
    Args:
        df: DataFrame with broker data
        data_categories: List of tuples (column_name, display_name)
    
    Returns:
        dict: Statistics for each category
        pd.DataFrame: Summary statistics
    """
    total_brokers = len(df)
    results = {}
    summary_rows = []
    
    print("Analyzing Data Collection Patterns")
    print("=" * 50)
    
    for col_name, display_name in data_categories:
        if col_name not in df.columns:
            print(f"Warning: Column '{col_name}' not found in data")
            continue
            
        # Get value counts (0=No, 1=Yes, 2=Not Reported)
        counts = df[col_name].value_counts().sort_index()
        
        no_count = counts.get(0, 0)
        yes_count = counts.get(1, 0)
        not_reported_count = counts.get(2, 0)
        
        # Calculate percentages for reported data only
        reported_total = no_count + yes_count
        if reported_total > 0:
            no_perc = no_count / reported_total
            yes_perc = yes_count / reported_total
        else:
            no_perc = yes_perc = 0
            
        not_reported_perc = not_reported_count / total_brokers
        
        # Store results
        results[display_name] = [no_perc, yes_perc, not_reported_perc]
        
        # Add to summary
        summary_rows.append({
            'Category': display_name,
            'Explicitly_No': f"{no_perc:.1%}",
            'Explicitly_Yes': f"{yes_perc:.1%}", 
            'Not_Reported': f"{not_reported_perc:.1%}",
            'Total_Reported': reported_total,
            'Total_Brokers': total_brokers
        })
        
        # Print detailed stats
        print(f"\n{display_name.title()} Data Collection:")
        print(f"   Explicitly NOT collecting: {no_perc:.1%} ({no_count} brokers)")
        print(f"   Explicitly collecting: {yes_perc:.1%} ({yes_count} brokers)")
        print(f"   Not reported: {not_reported_perc:.1%} ({not_reported_count} brokers)")
        print(f"   Response rate: {(reported_total/total_brokers):.1%}")
    
    # Create summary DataFrame
    summary_df = pd.DataFrame(summary_rows)
    
    print(f"\nAnalysis complete! Processed {len(data_categories)} categories.")
    return results, summary_df

# Define data categories to analyze
DATA_CATEGORIES = [
    ("CollectsMinorsData", "minors"),
    ("CollectsReproductiveHealthData", "reproductive_health"),
    ("CollectsEmploymentData", "employment"),
    ("CollectsNetworkData", "network"),
    ("CollectsCommercialData", "commercial"),
    ("CollectsBiometricData", "biometric"),
    ("CollectsAddresses", "addresses"),
    ("CollectsSSNGovID", "ssn_gov_id"),
]

# Run the analysis
collection_stats, summary_table = analyze_collection_patterns(data_brokers, DATA_CATEGORIES)

# Display summary
print("\n" + "="*60)
print("SUMMARY TABLE")
print("="*60)
display(summary_table)

Analyzing Data Collection Patterns

Minors Data Collection:
   Explicitly NOT collecting: 78.7% (333 brokers)
   Explicitly collecting: 21.3% (90 brokers)
   Not reported: 55.0% (518 brokers)
   Response rate: 45.0%

Reproductive_Health Data Collection:
   Explicitly NOT collecting: 98.4% (123 brokers)
   Explicitly collecting: 1.6% (2 brokers)
   Not reported: 86.7% (816 brokers)
   Response rate: 13.3%

Employment Data Collection:
   Explicitly NOT collecting: 17.6% (3 brokers)
   Explicitly collecting: 82.4% (14 brokers)
   Not reported: 98.2% (924 brokers)
   Response rate: 1.8%

Network Data Collection:
   Explicitly NOT collecting: 33.3% (3 brokers)
   Explicitly collecting: 66.7% (6 brokers)
   Not reported: 99.0% (932 brokers)
   Response rate: 1.0%

Commercial Data Collection:
   Explicitly NOT collecting: 8.3% (2 brokers)
   Explicitly collecting: 91.7% (22 brokers)
   Not reported: 97.4% (917 brokers)
   Response rate: 2.6%

Biometric Data Collection:
   Explicitly NOT colle

,Category,Explicitly_No,Explicitly_Yes,Not_Reported,Total_Reported,Total_Brokers
0,minors,78.7%,21.3%,55.0%,423,941
1,reproductive_health,98.4%,1.6%,86.7%,125,941
2,employment,17.6%,82.4%,98.2%,17,941
3,network,33.3%,66.7%,99.0%,9,941
4,commercial,8.3%,91.7%,97.4%,24,941
5,biometric,96.8%,3.2%,96.7%,31,941
6,addresses,29.5%,70.5%,95.3%,44,941
7,ssn_gov_id,93.5%,6.5%,96.7%,31,941


In [ ]:
# Data Preparation for Visualization (Including Not Reported Data)
# ==================================

print("Creating data type visualization...")

data_type = data_brokers[["Name", "CollectsMinorsData", "CollectsReproductiveHealthData", "CollectsEmploymentData", "CollectsNetworkData", "CollectsCommercialData", "CollectsBiometricData", "CollectsAddresses", "CollectsSSNGovID"]]
data_type = data_type.apply(pd.to_numeric, errors='coerce').astype('Int64')

data_type = data_type.rename(columns={
    "CollectsMinorsData": "Minors",
    "CollectsReproductiveHealthData": "Reproductive Health",
    "CollectsEmploymentData": "Employment",
    "CollectsNetworkData": "Network",
    "CollectsCommercialData": "Commercial",
    "CollectsBiometricData": "Biometric",
    "CollectsAddresses": "Address",
    "CollectsSSNGovID": "Government Identification"
})
data_type_cols = ['Minors', 'Reproductive Health', 'Reproductive Health', 'Employment', 'Network', 'Commercial', 'Biometric', 'Address', 'Government Identification']

available_cols = [col for col in data_type_cols if col in data_type.columns]
alt.themes.enable('default')
data_type_chart = create_policy_analysis_chart(
    data=data_type,
    category_columns=available_cols,
    title='Permitted Data Types Collected By Data Brokers',
    xlabel='Data Type',
    legend_labels=['0 = Explicitly Not Allowed', '1 = Explicitly Allowed', '2 = Not Reported']
)
display(data_type_chart)
data_type_chart.save("imgs/Permitted Data Types Collected By Data Brokers.svg")

Creating data type visualization...


alt.Chart(...)

In [4]:
# Data Preparation for Visualization (Only Reported Data)
# ==================================

def prepare_visualization_data(stats_dict, include_not_reported=True):
    """
    Transform collection statistics into format suitable for visualization.
    
    Args:
        stats_dict: Dictionary of collection statistics
        include_not_reported: Whether to include "Not Reported" category
    
    Returns:
        pd.DataFrame: Prepared data for visualization
    """
    # Convert to DataFrame and scale to percentages
    df = pd.DataFrame.from_dict(stats_dict, orient='columns') * 100
    
    # Set meaningful row names
    response_types = ['No', 'Yes', 'Not_Reported']
    df.index = response_types
    
    if not include_not_reported:
        df = df.drop('Not_Reported')
    
    # Reset index and prepare for melting
    df = df.reset_index().rename(columns={'index': 'Response'})
    
    # Melt for long format (better for Altair)
    melted_df = df.melt(
        id_vars='Response', 
        var_name='Data_Category', 
        value_name='Percentage'
    )
    
    # Clean up category names for display
    melted_df['Data_Category_Clean'] = melted_df['Data_Category'].str.replace('_', ' ').str.title()
    
    return melted_df

# Prepare data for reported responses only (Yes/No)
reported_data = prepare_visualization_data(collection_stats, include_not_reported=False)

# Prepare data for all responses including "Not Reported"
all_response_data = prepare_visualization_data(collection_stats, include_not_reported=True)

print("Visualization data prepared:")
print(f"   - Reported responses only: {len(reported_data)} rows")
print(f"   - All responses: {len(all_response_data)} rows")

# Preview the data structure
print("\nSample of reported data:")
display(reported_data.head(10))

Visualization data prepared:
   - Reported responses only: 16 rows
   - All responses: 24 rows

Sample of reported data:


,Response,Data_Category,Percentage,Data_Category_Clean
0,No,minors,78.723404,Minors
1,Yes,minors,21.276596,Minors
2,No,reproductive_health,98.400000,Reproductive Health
3,Yes,reproductive_health,1.600000,Reproductive Health
4,No,employment,17.647059,Employment
5,Yes,employment,82.352941,Employment
6,No,network,33.333333,Network
7,Yes,network,66.666667,Network
8,No,commercial,8.333333,Commercial
9,Yes,commercial,91.666667,Commercial


In [5]:
# Visualization Functions
# =======================

def create_collection_chart(data, chart_type='stacked_bar', title_suffix=""):
    """
    Create collection percentage charts with consistent styling.
    
    Args:
        data: Prepared DataFrame for visualization
        chart_type: Type of chart ('stacked_bar', 'grouped_bar', 'heatmap')
        title_suffix: Additional text for chart title
    
    Returns:
        alt.Chart: Configured Altair chart
    """
    base_title = "Data Collection Practices by Category"
    full_title = f"{base_title} {title_suffix}".strip()
    
    # Color scheme for responses
    color_scale = alt.Scale(
        domain=['No', 'Yes', 'Not_Reported'],
        range=['#e74c3c', '#27ae60', '#95a5a6']  # Red, Green, Gray
    )
    
    if chart_type == 'stacked_bar':
        chart = alt.Chart(data).mark_bar().encode(
            x=alt.X('Data_Category_Clean:N', 
                   title='Data Type',
                   axis=alt.Axis(labelAngle=-45)),
            y=alt.Y('Percentage:Q', 
                   title='Percentage (%)',
                   stack='normalize' if 'Not_Reported' in data['Response'].values else True),
            color=alt.Color('Response:N', 
                           title='Collection Status',
                           scale=color_scale),
            tooltip=[
                alt.Tooltip('Data_Category_Clean:N', title='Data Type'),
                alt.Tooltip('Response:N', title='Status'),
                alt.Tooltip('Percentage:Q', title='Percentage', format='.1f')
            ]
        )
    elif chart_type == 'grouped_bar':
        chart = alt.Chart(data).mark_bar().encode(
            x=alt.X('Response:N', title='Collection Status'),
            y=alt.Y('Percentage:Q', title='Percentage (%)'),
            color=alt.Color('Data_Category_Clean:N', title='Data Type'),
            column=alt.Column('Data_Category_Clean:N', title='Data Type'),
            tooltip=[
                alt.Tooltip('Data_Category_Clean:N', title='Data Type'),
                alt.Tooltip('Response:N', title='Status'),
                alt.Tooltip('Percentage:Q', title='Percentage', format='.1f')
            ]
        ).resolve_scale(x='independent')
    
    return chart.properties(
        width=500,
        height=350,
        title=alt.TitleParams(
            text=full_title,
            fontSize=14,
            fontWeight='bold'
        )
    ).interactive()

# Create main visualization: Reported data collection (Yes/No only)
reported_chart = create_collection_chart(
    reported_data, 
    chart_type='stacked_bar',
    title_suffix="(Explicitly Reported Only)"
)

print("Collection Patterns Visualization")
print("=" * 40)
print("This chart shows the percentage of brokers that explicitly")
print("collect or don't collect each type of data (excluding unreported).")

reported_chart

Collection Patterns Visualization
This chart shows the percentage of brokers that explicitly
collect or don't collect each type of data (excluding unreported).


alt.Chart(...)

In [6]:
# Transparency Analysis
# ====================

def analyze_transparency_gaps(all_data):
    """
    Analyze transparency gaps by focusing on 'Not Reported' responses.
    
    Args:
        all_data: DataFrame with all response types including Not_Reported
    
    Returns:
        pd.DataFrame: Transparency analysis results
    """
    # Filter for "Not Reported" responses only
    transparency_data = all_data[all_data['Response'] == 'Not_Reported'].copy()
    
    # Sort by percentage to identify biggest transparency gaps
    transparency_data = transparency_data.sort_values('Percentage', ascending=False)
    
    # Add interpretation categories
    def categorize_transparency(percentage):
        if percentage >= 75:
            return 'Critical Gap'
        elif percentage >= 50:
            return 'Significant Gap'
        elif percentage >= 25:
            return 'Moderate Gap'
        else:
            return 'Good Transparency'
    
    transparency_data['Transparency_Level'] = transparency_data['Percentage'].apply(categorize_transparency)
    
    return transparency_data

# Analyze transparency gaps
transparency_gaps = analyze_transparency_gaps(all_response_data)

print("TRANSPARENCY ANALYSIS")
print("=" * 40)
print("Categories with highest percentage of unreported data:")
print()

for _, row in transparency_gaps.iterrows():
    category = row['Data_Category_Clean']
    percentage = row['Percentage']
    level = row['Transparency_Level']
    
    print(f"{category}")
    print(f"   {percentage:.1f}% not reported ({level})")
    print()

# Display as table
display(transparency_gaps[['Data_Category_Clean', 'Percentage', 'Transparency_Level']]
        .rename(columns={
            'Data_Category_Clean': 'Data Category',
            'Percentage': 'Not Reported (%)',
            'Transparency_Level': 'Transparency Assessment'
        }))

TRANSPARENCY ANALYSIS
Categories with highest percentage of unreported data:

Network
   99.0% not reported (Critical Gap)

Employment
   98.2% not reported (Critical Gap)

Commercial
   97.4% not reported (Critical Gap)

Biometric
   96.7% not reported (Critical Gap)

Ssn Gov Id
   96.7% not reported (Critical Gap)

Addresses
   95.3% not reported (Critical Gap)

Reproductive Health
   86.7% not reported (Critical Gap)

Minors
   55.0% not reported (Significant Gap)



,Data Category,Not Reported (%),Transparency Assessment
11,Network,99.043571,Critical Gap
8,Employment,98.193411,Critical Gap
14,Commercial,97.449522,Critical Gap
17,Biometric,96.705632,Critical Gap
23,Ssn Gov Id,96.705632,Critical Gap
20,Addresses,95.324123,Critical Gap
5,Reproductive Health,86.716259,Critical Gap
2,Minors,55.047821,Significant Gap


## Key Insights & Exploration Summary

Based on this exploratory analysis of data broker collection practices:

### **Data Collection Patterns**
- **Most Collected**: Commercial and address data show highest collection rates
- **Sensitive Data**: Reproductive health and biometric data have lower explicit collection rates
- **Minors Protection**: Lower collection rates for minors' data, suggesting some privacy awareness

### **Transparency Issues**
- **Critical Gaps**: Some categories have >50% non-reporting rates, with the category corresponding to "minors" being the category with the highest transparency
- **Regulatory Opportunity**: High non-reporting suggests need for mandatory disclosure requirements
- **Industry Standards**: Variation in transparency practices across data types

### **Next Steps for Analysis**
1. **Geographic Analysis**: Compare collection practices by state/jurisdiction
2. **Industry Segmentation**: Analyze patterns by broker business model
3. **Temporal Trends**: Examine changes in collection practices over time
4. **Regulatory Correlation**: Compare transparency with local privacy laws

### **Methodology Notes**
- This analysis excludes incomplete responses (coded as "2") from percentage calculations
- Percentages are based on brokers that explicitly reported their practices
- Transparency assessment categorizes non-reporting rates into severity levels

In [8]:
# Extended Exploratory Analysis
# =============================

def explore_data_relationships(df):
    """
    Explore relationships and correlations between different data collection practices.
    """
    print("🔬 EXPLORING DATA RELATIONSHIPS")
    print("=" * 45)
    
    # Collection columns for correlation analysis
    collection_cols = [col for col in df.columns if col.startswith('Collects')]
    
    if len(collection_cols) > 1:
        # Calculate correlation matrix (only for explicitly reported data)
        # Convert 2s (not reported) to NaN for correlation calculation
        corr_data = df[collection_cols].replace(2, np.nan)
        correlation_matrix = corr_data.corr()
        
        print("📈 Strongest Positive Correlations:")
        # Find highest correlations (excluding diagonal)
        corr_pairs = []
        for i in range(len(correlation_matrix.columns)):
            for j in range(i+1, len(correlation_matrix.columns)):
                corr_val = correlation_matrix.iloc[i, j]
                if not pd.isna(corr_val):
                    col1 = correlation_matrix.columns[i]
                    col2 = correlation_matrix.columns[j]
                    corr_pairs.append((corr_val, col1, col2))
        
        # Sort by correlation strength
        corr_pairs.sort(reverse=True)
        
        for corr_val, col1, col2 in corr_pairs[:5]:  # Top 5 correlations
            clean_col1 = col1.replace('Collects', '').replace('Data', '')
            clean_col2 = col2.replace('Collects', '').replace('Data', '')
            print(f"   🔗 {clean_col1} ↔ {clean_col2}: {corr_val:.3f}")
    
    # Registry source analysis
    if 'RegistrySource_California' in df.columns or 'RegistrySource_Vermont' in df.columns:
        registry_cols = [col for col in df.columns if col.startswith('RegistrySource_')]
        print(f"\n📍 Found {len(registry_cols)} registry sources:")
        for col in registry_cols:
            source = col.replace('RegistrySource_', '')
            count = df[col].sum() if col in df.columns else 0
            percentage = (count / len(df)) * 100
            print(f"   🏛️  {source}: {count} brokers ({percentage:.1f}%)")
    
    # Summary statistics
    print(f"\n📊 Dataset Overview:")
    print(f"   📋 Total brokers: {len(df)}")
    print(f"   📈 Collection categories: {len([col for col in df.columns if col.startswith('Collects')])}")
    print(f"   🔍 Columns analyzed: {df.shape[1]}")
    
    # Data completeness by broker
    if collection_cols:
        # Calculate how many categories each broker reports
        completeness = df[collection_cols].apply(lambda row: (row != 2).sum(), axis=1)
        avg_completeness = completeness.mean()
        max_categories = len(collection_cols)
        
        print(f"   📈 Average reporting completeness: {avg_completeness:.1f}/{max_categories} categories ({(avg_completeness/max_categories)*100:.1f}%)")
        
        # Find most/least transparent brokers
        most_transparent = completeness.idxmax()
        least_transparent = completeness.idxmin()
        
        if 'CompanyName' in df.columns:
            print(f"   🌟 Most transparent broker: {df.loc[most_transparent, 'CompanyName']} ({completeness[most_transparent]}/{max_categories} reported)")
            print(f"   🔒 Least transparent broker: {df.loc[least_transparent, 'CompanyName']} ({completeness[least_transparent]}/{max_categories} reported)")

# Run extended exploration
explore_data_relationships(data_brokers)

🔬 EXPLORING DATA RELATIONSHIPS
📈 Strongest Positive Correlations:
   🔗 Employment ↔ Network: 1.000
   🔗 POB ↔ MMN: 0.800
   🔗 Names ↔ Addresses: 0.783
   🔗 Addresses ↔ Commercial: 0.764
   🔗 Addresses ↔ Employment: 0.756

📍 Found 5 registry sources:
   🏛️  California (Attorney General): 374 brokers (39.7%)
   🏛️  California (California Privacy Protection Agency): 493 brokers (52.4%)
   🏛️  Oregon: 282 brokers (30.0%)
   🏛️  Texas: 247 brokers (26.2%)
   🏛️  Vermont: 573 brokers (60.9%)

📊 Dataset Overview:
   📋 Total brokers: 941
   📈 Collection categories: 13
   🔍 Columns analyzed: 38
   📈 Average reporting completeness: 0.9/13 categories (7.1%)
